<a href="https://colab.research.google.com/github/Faraz-Khan02/Book-Recommendation-System/blob/main/Book_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Name**    - **Book Recommendation System**



##### **Project Type**    - Unsupervised
##### **Contribution**    - Individual
##### **Name**   -     Faraz Faisal Khan


# **Project Summary -**

Write the summary here within 500-600 words.

# **GitHub Link -**

Provide your GitHub Link here.

# **Problem Statement**


**During the last few decades, with the rise of Youtube, Amazon, Netflix, and many other such web services, recommender systems have taken more and more place in our lives. From e-commerce (suggest to buyers articles that could interest them) to online advertisement (suggest to users the right contents, matching their preferences), recommender systems are today unavoidable in our daily online journeys.
In a very general way, recommender systems are algorithms aimed at suggesting relevant items to users (items being movies to watch, text to read, products to buy, or anything else depending on industries).
Recommender systems are really critical in some industries as they can generate a huge amount of income when they are efficient or also be a way to stand out significantly from competitors. The main objective is to create a book recommendation system for users.**

# ***Let's Begin !***

### Import Libraries

In [ ]:
# Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings("ignore")

### Dataset Loading

In [ ]:
# Mounting Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Load Dataset
books = pd.read_csv("/content/drive/MyDrive/Capstone Project - 4/Books.csv")
users = pd.read_csv("/content/drive/MyDrive/Capstone Project - 4/Users.csv")
ratings = pd.read_csv("/content/drive/MyDrive/Capstone Project - 4/Ratings.csv")

## **Books Data**

In [ ]:
# Books data first look
books.head()

Here, Books data contains many features which are as follows:


*   **ISBN** : It contains ISBN number of the books which mean  International Standard Book Number.
*   **Book-Title** : It contains title of the books.


*   **Book-Author** : It contains the name of author of that book.
*   **Year-Of-Publication** : It contains the year in which that book was published.


*   **Publisher** : It contains the name of the publisher.
*   **Image-URL-S** : It contains the Image Url of small size.


*   **Image-URL-M** : It contains the Image Url of medium size.
*   **Image-URL-L** : It contains the Image Url of Large size.









In [ ]:
# Shape of Books data
books.shape

Our, Books dataset contains 271360 rows and 8 columns.

In [ ]:
# Books dataset info
books.info

In [ ]:
# Books Dataset Duplicate Value Count
duplicate = books.duplicated()
print(duplicate.value_counts())

It means there is no any duplicate values in the books dataset.

In [ ]:
# Missing Values/Null Values Count
books.isnull().sum()

## ***Data Cleaning of Books Dataset***

Here, Image-URL-S, Image-URL-M, Image-URL-L are not any useful feature for our recommmendation so we will drop it.

In [ ]:
# dropping last three columns 
books.drop(['Image-URL-S', 'Image-URL-M', 'Image-URL-L'],axis=1,inplace=True)

In [ ]:
# After dropping 3 columns
books.head()

In [ ]:
# Checking data types of columns
print(books.dtypes)

Here, Year-Of-Publication should have integer datatype but its object so we will check the unique values of it.

In [ ]:
#Get the unique values of Year-Of-Publication	
books['Year-Of-Publication'].unique()

Here we can see that 'DK Publishing Inc' and 'Gallimard' are wrong entry and we can see Year of Publication is more than 2004 which is wrong because our data was published in 2004.

In [ ]:
#Checking the rows having 'DK Publishing Inc' and 'Gallimard' as Year-Of-Publication
books.loc[(books['Year-Of-Publication'] == 'DK Publishing Inc') |( books['Year-Of-Publication'] == 'Gallimard'),:]

Here, we can clearly see that Book-Author and Year-Of-Publication is mismatched so we will replace it correctly.

In [ ]:
# Correcting 1st row 
books.loc[books.ISBN == '0789466953','Year-Of-Publication'] = 2000
books.loc[books.ISBN == '0789466953','Book-Author'] = "James Buckley"
books.loc[books.ISBN == '0789466953','Publisher'] = "DK Publishing Inc"
books.loc[books.ISBN == '0789466953','Book-Title'] = "DK Readers: Creating the X-Men, How Comic Books Come to Life (Level 4: Proficient Readers)"

In [ ]:
# Correcting 2nd row
books.loc[books.ISBN == '2070426769','Year-Of-Publication'] = 2003
books.loc[books.ISBN == '2070426769','Book-Author'] = "Jean-Marie Gustave Le ClÃ?Â©zio"
books.loc[books.ISBN == '2070426769','Publisher'] = "Gallimard"
books.loc[books.ISBN == '2070426769','Book-Title'] = "Peuple du ciel, suivi de 'Les Bergers"

In [ ]:
# Correcting 3rd row
books.loc[books.ISBN == '078946697X','Year-Of-Publication'] = 2000
books.loc[books.ISBN == '078946697X','Book-Author'] = "Michael Teitelbaum"
books.loc[books.ISBN == '078946697X','Publisher'] = "DK Publishing Inc"
books.loc[books.ISBN == '078946697X','Book-Title'] = "DK Readers: Creating the X-Men, How It All Began (Level 4: Proficient Readers)"

In [ ]:
#Rechecking after correcting
books.loc[(books.ISBN == '0789466953') | (books.ISBN == '078946697X') | (books.ISBN == '2070426769'),:]

In [ ]:
# Converting year of publication to type int
books['Year-Of-Publication'] = books['Year-Of-Publication'].astype(int)

In [ ]:
# Checking year in sorted manner
sorted(books['Year-Of-Publication'].unique())

Here, 0 is incorrect and our data was of 2004 and Year-Of-Publication is greater than 2004 which means it is wrongly interpreted.So, we will interpret those values with NaN. 

In [ ]:
# Removing year 0 and years above 2004 with NaN
books.loc[(books['Year-Of-Publication'] > 2004) | (books['Year-Of-Publication']==0),'Year-Of-Publication'] = np.NAN

## ***Handling Missing Values***

In [ ]:
#Checking for missing values
books.isnull().sum()

Here, we can see Book Author contains '1' missing values , Year-Of-Publication contain 4690 missing values and Publisher contains '2' missing values.

**Conclusion**

Write the conclusion here.

### ***Hurrah! You have successfully completed your Machine Learning Capstone Project !!!***